In [61]:
# import os

# from google.colab import drive
# drive.mount('/content/drive')

In [62]:
import re

# 1. Extract PDF

In [63]:
pip install pdfplumber

In [64]:
pip install pandas

In [65]:
import os
import pdfplumber
import pandas as pd

def extract_pdf_content(file_path):
    # 讀取 PDF 文件
    with pdfplumber.open(file_path) as pdf:
        # 初始化 DataFrame
        df = pd.DataFrame(columns=['File Name', 'Text'])

        # 提取整個 PDF 的文本
        all_text = ''
        for page in pdf.pages:
            all_text += (page.extract_text() +'\n')

        # 將文件名和文本添加到 DataFrame
        #df = df.append({'File Name': os.path.basename(file_path), 'Text': all_text}, ignore_index=True)
        df = pd.concat([df, pd.DataFrame({'File Name': [os.path.basename(file_path)], 'Text': [all_text]})], ignore_index=True)

    return df

def batch_extract_pdf_content(folder_path):
    # 初始化空的 DataFrame
    batch_df = pd.DataFrame(columns=['File Name', 'Text'])

    # 列出文件夾中的所有 PDF 文件
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

    # 遍歷每個 PDF 文件並調用提取函數
    for pdf_file in pdf_files:
        file_path = os.path.join(folder_path, pdf_file)
        pdf_df = extract_pdf_content(file_path)

        # 將當前 PDF 文件的內容添加到批處理 DataFrame 中
        batch_df = pd.concat([batch_df, pdf_df], ignore_index=True)

    return batch_df

In [66]:
# 調用批量處理函數並將 PDF 內容存入 DataFrame
pdf_folder_path = r"D:/gpa_search_db/pdf" #'/content/drive/MyDrive/大學/成績分布/'
batch_df = batch_extract_pdf_content(pdf_folder_path)

# 顯示 DataFrame 的內容
#print(batch_df.head())

In [67]:
batch_df

,File Name,Text
0,B10833021-1081.pdf,國立臺灣科技大學\n1081 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...
1,B10833021-1082.pdf,國立臺灣科技大學\n1082 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...
2,B10833021-1091.pdf,國立臺灣科技大學\n1091 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...
3,B10833021-1092.pdf,國立臺灣科技大學\n1092 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...
4,B10833021-1101.pdf,國立臺灣科技大學\n1101 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...
5,B10833021-1102.pdf,國立臺灣科技大學\n1102 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...
6,B10833021-1111.pdf,國立臺灣科技大學\n1111 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...


# 2. Data Processing

## split 換行

In [68]:
df = batch_df

In [69]:
# 定義一個函數來對每行文本進行分割，並返回分割後的結果
def split_text(row):
    return row.split('\n')

# 創建一個空的列表，用於存儲分割後的文本數據
split_data = []

# 在 'Text' 列的每一行應用 split_text 函數，並將結果添加到 split_data 中
for index, row in batch_df.iterrows():
    split_result = split_text(row['Text'])
    if len(split_result) > 7:  # 檢查是否已經到達第 7 行
        split_result = split_result[:7] + ['\n'.join(split_result[7:])]

    # 將分割後的結果添加到 split_data 列表中
    split_data.append(split_result)

# 將分割後的數據列表轉換為 DataFrame
split_df = pd.DataFrame(split_data)

# 給新 DataFrame 添加列名
split_df.columns = [f'Column_{i+1}' for i in range(split_df.shape[1])]

# 將原始數據和分割後的數據合並成一個新的 DataFrame
merged_df = pd.concat([df, split_df], axis=1)

# 顯示結果
merged_df


,File Name,Text,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8
0,B10833021-1081.pdf,國立臺灣科技大學\n1081 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1081 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理一 列印日期： 2020年6月28日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% --...
1,B10833021-1082.pdf,國立臺灣科技大學\n1082 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1082 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理二 列印日期： 2020年9月16日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 CC1255304 3 A+ 百分比 18.1% 29.9% 5.6% .6% 2.3%...
2,B10833021-1091.pdf,國立臺灣科技大學\n1091 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1091 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理二 列印日期： 2021年3月16日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 CC1011301 0 通過 百分比 未通過\n校定英文能力檢測 (人數) 548 54...
3,B10833021-1092.pdf,國立臺灣科技大學\n1092 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1092 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理二 列印日期： 2021年7月26日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 BA305B001 3 A+ 百分比 55.2% 13.4% 4.5% 1.5% -- ...
4,B10833021-1101.pdf,國立臺灣科技大學\n1101 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1101 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理三 列印日期： 2022年6月22日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 B CEG321301 2 A+ 百分比 34.3% 25.9% 5.4% .6% --...
5,B10833021-1102.pdf,國立臺灣科技大學\n1102 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1102 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理三 列印日期： 2022年7月8日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 B IBG006301 2 A+ 百分比 76.5% 5.5% 3.3% 1.1% .5...
6,B10833021-1111.pdf,國立臺灣科技大學\n1111 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1111 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理四 列印日期： 2023年2月21日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 BA2505701 3 A+ 百分比 58.3% 33.3% -- -- -- -- -...


In [70]:
# 合並 Column_4 到 Column_7 的內容為一個字符串
merged_titles = ' '.join(merged_df.iloc[0, 5:9])
print(len(merged_titles))

# 使用空格拆分合並後的字符串，生成新的列標題
split_titles = merged_titles.split()
print("split_titles", split_titles)

# 確保新的列標題與 DataFrame 的列數匹配
#while len(split_titles) < len(merged_df.columns):
    #split_titles.append('')

# 將拆分後的列標題賦值給 DataFrame 的列名
#merged_df.columns = split_titles

# 刪除第一行，因為已經使用內容作為列標題了
#merged_df = merged_df.drop(0)

# 添加新行並將其內容填充為空白
#merged_df = merged_df.append(pd.Series([''] * len(merged_df.columns), index=merged_df.columns), ignore_index=True)

#merged_df


142
split_titles ['序', '通識', '課程代碼', '學分數', '成績', '90-100', '80~84', '73~76', '67~69', '60~62', '1~49', '成績', '總計', '向度', 'A+', 'A-', 'B', 'C+', 'C-', 'E', '未到', 'Total', '課程名稱', '備註說明', '85~89', '77~79', '70~72', '63~66', '50~59', '0', 'A', 'B+', 'B-', 'C', 'D', 'X']


## 處理標題 (欄位名稱)

In [71]:
#split_titles ['序', '通識', '課程代碼', '學分數', '成績', '90-100', '80~84', '73~76', '67~69', '60~62', '1~49', '成績', '總計', '向度', 'A+', 'A-', 'B', 'C+', 'C-', 'E', '未到', 'Total', '課程名稱', '備註說明', '85~89', '77~79', '70~72', '63~66', '50~59', '0', 'A', 'B+', 'B-', 'C', 'D', 'X']

In [72]:
#
title = []
grade = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D', 'E', 'X', '成績未到']

title.append("學期&學年")
title.append(split_titles[2])
title.append(split_titles[22])
title.append(split_titles[3])
title.append("人數/百分比")
title.extend(grade)
title.append(split_titles[21])

print("title:", title)

title: ['學期&學年', '課程代碼', '課程名稱', '學分數', '人數/百分比', 'A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D', 'E', 'X', '成績未到', 'Total']


In [73]:
title[0]

'學期&學年'

In [74]:
# 添加新的列名，這里假設 sorted_merged_titles 是一個列表，包含要添加的列名
for col_name in title:
    # 使用 insert 方法在 DataFrame 中添加新的列名，默認將其內容設為空字符串
    merged_df.insert(len(merged_df.columns), col_name, '')

# 打印修改後的 DataFrame
merged_df

,File Name,Text,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,...,B,B-,C+,C,C-,D,E,X,成績未到,Total
0,B10833021-1081.pdf,國立臺灣科技大學\n1081 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1081 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理一 列印日期： 2020年6月28日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% --...,...,,,,,,,,,,
1,B10833021-1082.pdf,國立臺灣科技大學\n1082 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1082 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理二 列印日期： 2020年9月16日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 CC1255304 3 A+ 百分比 18.1% 29.9% 5.6% .6% 2.3%...,...,,,,,,,,,,
2,B10833021-1091.pdf,國立臺灣科技大學\n1091 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1091 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理二 列印日期： 2021年3月16日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 CC1011301 0 通過 百分比 未通過\n校定英文能力檢測 (人數) 548 54...,...,,,,,,,,,,
3,B10833021-1092.pdf,國立臺灣科技大學\n1092 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1092 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理二 列印日期： 2021年7月26日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 BA305B001 3 A+ 百分比 55.2% 13.4% 4.5% 1.5% -- ...,...,,,,,,,,,,
4,B10833021-1101.pdf,國立臺灣科技大學\n1101 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1101 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理三 列印日期： 2022年6月22日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 B CEG321301 2 A+ 百分比 34.3% 25.9% 5.4% .6% --...,...,,,,,,,,,,
5,B10833021-1102.pdf,國立臺灣科技大學\n1102 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1102 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理三 列印日期： 2022年7月8日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 B IBG006301 2 A+ 百分比 76.5% 5.5% 3.3% 1.1% .5...,...,,,,,,,,,,
6,B10833021-1111.pdf,國立臺灣科技大學\n1111 成績分布表\n學號： B10833021 姓名： 謝昕恬 班級...,國立臺灣科技大學,1111 成績分布表,學號： B10833021 姓名： 謝昕恬 班級： 四管理四 列印日期： 2023年2月21日,序 通識 課程代碼 學分數 成績 90-100 80~84 73~76 67~69 60~6...,向度 A+ A- B C+ C- E 未到 Total,課程名稱 備註說明 85~89 77~79 70~72 63~66 50~59 0,A B+ B- C D X,1 BA2505701 3 A+ 百分比 58.3% 33.3% -- -- -- -- -...,...,,,,,,,,,,


## 刪除不需要的行

In [75]:
merged_df.iloc[0,0][0:4]

'B108'

In [76]:
# 提取學年度: 提取 Column_2 的前四個字符並替換為新列
merged_df['學期&學年'] = merged_df['Column_2'].str.slice(stop=4)
#merged_df

In [77]:
merged_df = merged_df.drop(['File Name','Text','Column_1','Column_2','Column_3','Column_4','Column_5','Column_6','Column_7'], axis=1)
merged_df

,Column_8,學期&學年,課程代碼,課程名稱,學分數,人數/百分比,A+,A,A-,B+,B,B-,C+,C,C-,D,E,X,成績未到,Total
0,1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% --...,1081,,,,,,,,,,,,,,,,,,
1,1 CC1255304 3 A+ 百分比 18.1% 29.9% 5.6% .6% 2.3%...,1082,,,,,,,,,,,,,,,,,,
2,1 CC1011301 0 通過 百分比 未通過\n校定英文能力檢測 (人數) 548 54...,1091,,,,,,,,,,,,,,,,,,
3,1 BA305B001 3 A+ 百分比 55.2% 13.4% 4.5% 1.5% -- ...,1092,,,,,,,,,,,,,,,,,,
4,1 B CEG321301 2 A+ 百分比 34.3% 25.9% 5.4% .6% --...,1101,,,,,,,,,,,,,,,,,,
5,1 B IBG006301 2 A+ 百分比 76.5% 5.5% 3.3% 1.1% .5...,1102,,,,,,,,,,,,,,,,,,
6,1 BA2505701 3 A+ 百分比 58.3% 33.3% -- -- -- -- -...,1111,,,,,,,,,,,,,,,,,,


## 處理row (不同課程)


### 區隔課程資料

In [78]:
#pd.set_option('display.max_colwidth', -1)
#merged_df.iloc[0,10]
merged_df['Column_8'][0]

'1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% -- -- 100%\n新生中文說寫專題 (人數) 15 27 1 2 2 0 0 75\n百分比 21.3% 12% 4% -- -- --\n(人數) 16 9 3 0 0 0\n2 FE1851702 2 A 百分比 -- 57.1% -- -- -- -- -- 100%\n國際議題分析 (人數) 0 16 0 0 0 0 0 28\n百分比 32.1% -- -- 3.6% 3.6% 3.6%\n(人數) 9 0 0 1 1 1\n3 E GE3606301 2 B- 百分比 45.1% 25.5% 3.9% 2% 2% -- -- 100%\n民法概要 (人數) 23 13 2 1 1 0 0 51\n百分比 3.9% -- 9.8% 2% 5.9% --\n(人數) 2 0 5 1 3 0\n4 MB112A001 3 A- 百分比 2.6% 52.6% 2.6% -- -- -- -- 100%\n經濟學(上) (人數) 1 20 1 0 0 0 0 38\n百分比 34.2% 7.9% -- -- -- --\n(人數) 13 3 0 0 0 0\n5 MB1200301 3 A+ 百分比 77.5% 2.5% -- -- -- -- -- 100%\n計算機概論 (人數) 31 1 0 0 0 0 0 40\n百分比 20% -- -- -- -- --\n(人數) 8 0 0 0 0 0\n6 MB1600301 3 A 百分比 48.6% 8.1% 2.7% 2.7% -- -- -- 100%\n微積分 (人數) 18 3 1 1 0 0 0 37\n百分比 27% 5.4% 2.7% -- 2.7% --\n(人數) 10 2 1 0 1 0\n7 MB200A001 3 B- 百分比 6.3% 16.7% 16.7% 4.2% 2.1% 2.1% -- 100%\n會計學(上) (人數) 3 8 8 2 1 1 0 48\n百分比 16.7% 6.3% 25% 4.2% -- --\n(人數) 8 3 12 2 0 0\n8 MB2100301 3 A+ 百分比 20.3% 18.8% 10.1% 1.4% 13% -- -- 100%\n管理與企

In [79]:
# 確認 segments 以 match 形式可以找到 (數字搭配大寫英文字母:找到課號)
#for s in segments:
 # if re.match(r'\d+\s+[A-Z]', s):
  #      print(s)

In [80]:
segments = merged_df['Column_8'][0].split('\n')

# 儲存全部課程
grouped_segments = []
# 儲存單一課程
group = []

for segment in segments:
    # 使用正則表達式匹配數字後緊跟一個大寫字母單詞的位置進行分割
    if re.match(r'\d+\s+[A-Z]', segment):
        grouped_segments.append(group)
        group = []  # 重置group為空列表，準備存儲下一組segments
        #print(group)
   # elif re.match(r'(?<!\d)\d{1,2}\s+[A-Z]{1,2}', segment):
    #    grouped_segments.append(group)
     #   group = []

    group.append(segment)


# 將剩餘的segments加入grouped_segments中
if group:
    grouped_segments.append(group)

# 打印合並後的內容
#for group in grouped_segments:
 #   for segment in group:
  #      print(segment)
        #print('....................')
   # print('-----------------')
grouped_segments

[[],
 ['1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% -- -- 100%',
  '新生中文說寫專題 (人數) 15 27 1 2 2 0 0 75',
  '百分比 21.3% 12% 4% -- -- --',
  '(人數) 16 9 3 0 0 0'],
 ['2 FE1851702 2 A 百分比 -- 57.1% -- -- -- -- -- 100%',
  '國際議題分析 (人數) 0 16 0 0 0 0 0 28',
  '百分比 32.1% -- -- 3.6% 3.6% 3.6%',
  '(人數) 9 0 0 1 1 1'],
 ['3 E GE3606301 2 B- 百分比 45.1% 25.5% 3.9% 2% 2% -- -- 100%',
  '民法概要 (人數) 23 13 2 1 1 0 0 51',
  '百分比 3.9% -- 9.8% 2% 5.9% --',
  '(人數) 2 0 5 1 3 0'],
 ['4 MB112A001 3 A- 百分比 2.6% 52.6% 2.6% -- -- -- -- 100%',
  '經濟學(上) (人數) 1 20 1 0 0 0 0 38',
  '百分比 34.2% 7.9% -- -- -- --',
  '(人數) 13 3 0 0 0 0'],
 ['5 MB1200301 3 A+ 百分比 77.5% 2.5% -- -- -- -- -- 100%',
  '計算機概論 (人數) 31 1 0 0 0 0 0 40',
  '百分比 20% -- -- -- -- --',
  '(人數) 8 0 0 0 0 0'],
 ['6 MB1600301 3 A 百分比 48.6% 8.1% 2.7% 2.7% -- -- -- 100%',
  '微積分 (人數) 18 3 1 1 0 0 0 37',
  '百分比 27% 5.4% 2.7% -- 2.7% --',
  '(人數) 10 2 1 0 1 0'],
 ['7 MB200A001 3 B- 百分比 6.3% 16.7% 16.7% 4.2% 2.1% 2.1% -- 100%',
  '會計學(上) (人數) 3 8 8 2 1 1 0 48',


In [81]:
# 去除逗號
grouped_grades = ([' '.join(segment)  for segment in grouped_segments])
# 以空白分隔
grouped_grades = [segment.split() for segment in grouped_grades]
print(grouped_grades)

[[], ['1', 'CC1257303', '3', 'A+', '百分比', '20%', '36%', '1.3%', '2.7%', '2.7%', '--', '--', '100%', '新生中文說寫專題', '(人數)', '15', '27', '1', '2', '2', '0', '0', '75', '百分比', '21.3%', '12%', '4%', '--', '--', '--', '(人數)', '16', '9', '3', '0', '0', '0'], ['2', 'FE1851702', '2', 'A', '百分比', '--', '57.1%', '--', '--', '--', '--', '--', '100%', '國際議題分析', '(人數)', '0', '16', '0', '0', '0', '0', '0', '28', '百分比', '32.1%', '--', '--', '3.6%', '3.6%', '3.6%', '(人數)', '9', '0', '0', '1', '1', '1'], ['3', 'E', 'GE3606301', '2', 'B-', '百分比', '45.1%', '25.5%', '3.9%', '2%', '2%', '--', '--', '100%', '民法概要', '(人數)', '23', '13', '2', '1', '1', '0', '0', '51', '百分比', '3.9%', '--', '9.8%', '2%', '5.9%', '--', '(人數)', '2', '0', '5', '1', '3', '0'], ['4', 'MB112A001', '3', 'A-', '百分比', '2.6%', '52.6%', '2.6%', '--', '--', '--', '--', '100%', '經濟學(上)', '(人數)', '1', '20', '1', '0', '0', '0', '0', '38', '百分比', '34.2%', '7.9%', '--', '--', '--', '--', '(人數)', '13', '3', '0', '0', '0', '0'], ['5', 'MB1200301', '3

In [82]:
len(grouped_grades)

11

In [83]:
merged_df

,Column_8,學期&學年,課程代碼,課程名稱,學分數,人數/百分比,A+,A,A-,B+,B,B-,C+,C,C-,D,E,X,成績未到,Total
0,1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% --...,1081,,,,,,,,,,,,,,,,,,
1,1 CC1255304 3 A+ 百分比 18.1% 29.9% 5.6% .6% 2.3%...,1082,,,,,,,,,,,,,,,,,,
2,1 CC1011301 0 通過 百分比 未通過\n校定英文能力檢測 (人數) 548 54...,1091,,,,,,,,,,,,,,,,,,
3,1 BA305B001 3 A+ 百分比 55.2% 13.4% 4.5% 1.5% -- ...,1092,,,,,,,,,,,,,,,,,,
4,1 B CEG321301 2 A+ 百分比 34.3% 25.9% 5.4% .6% --...,1101,,,,,,,,,,,,,,,,,,
5,1 B IBG006301 2 A+ 百分比 76.5% 5.5% 3.3% 1.1% .5...,1102,,,,,,,,,,,,,,,,,,
6,1 BA2505701 3 A+ 百分比 58.3% 33.3% -- -- -- -- -...,1111,,,,,,,,,,,,,,,,,,


### 合併處理所有檔案的值

In [84]:
grade_df = merged_df
grade_df
#grouped_grades = [segment for segment in grouped_grades if len(segment) >= 37]

,Column_8,學期&學年,課程代碼,課程名稱,學分數,人數/百分比,A+,A,A-,B+,B,B-,C+,C,C-,D,E,X,成績未到,Total
0,1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% --...,1081,,,,,,,,,,,,,,,,,,
1,1 CC1255304 3 A+ 百分比 18.1% 29.9% 5.6% .6% 2.3%...,1082,,,,,,,,,,,,,,,,,,
2,1 CC1011301 0 通過 百分比 未通過\n校定英文能力檢測 (人數) 548 54...,1091,,,,,,,,,,,,,,,,,,
3,1 BA305B001 3 A+ 百分比 55.2% 13.4% 4.5% 1.5% -- ...,1092,,,,,,,,,,,,,,,,,,
4,1 B CEG321301 2 A+ 百分比 34.3% 25.9% 5.4% .6% --...,1101,,,,,,,,,,,,,,,,,,
5,1 B IBG006301 2 A+ 百分比 76.5% 5.5% 3.3% 1.1% .5...,1102,,,,,,,,,,,,,,,,,,
6,1 BA2505701 3 A+ 百分比 58.3% 33.3% -- -- -- -- -...,1111,,,,,,,,,,,,,,,,,,


In [85]:
grade_df.shape[0]

7

In [86]:
print(grouped_grades)

[[], ['1', 'CC1257303', '3', 'A+', '百分比', '20%', '36%', '1.3%', '2.7%', '2.7%', '--', '--', '100%', '新生中文說寫專題', '(人數)', '15', '27', '1', '2', '2', '0', '0', '75', '百分比', '21.3%', '12%', '4%', '--', '--', '--', '(人數)', '16', '9', '3', '0', '0', '0'], ['2', 'FE1851702', '2', 'A', '百分比', '--', '57.1%', '--', '--', '--', '--', '--', '100%', '國際議題分析', '(人數)', '0', '16', '0', '0', '0', '0', '0', '28', '百分比', '32.1%', '--', '--', '3.6%', '3.6%', '3.6%', '(人數)', '9', '0', '0', '1', '1', '1'], ['3', 'E', 'GE3606301', '2', 'B-', '百分比', '45.1%', '25.5%', '3.9%', '2%', '2%', '--', '--', '100%', '民法概要', '(人數)', '23', '13', '2', '1', '1', '0', '0', '51', '百分比', '3.9%', '--', '9.8%', '2%', '5.9%', '--', '(人數)', '2', '0', '5', '1', '3', '0'], ['4', 'MB112A001', '3', 'A-', '百分比', '2.6%', '52.6%', '2.6%', '--', '--', '--', '--', '100%', '經濟學(上)', '(人數)', '1', '20', '1', '0', '0', '0', '0', '38', '百分比', '34.2%', '7.9%', '--', '--', '--', '--', '(人數)', '13', '3', '0', '0', '0', '0'], ['5', 'MB1200301', '3

In [87]:
for row in range(0, grade_df.shape[0]):
  # 找出單一課程，並分別存入 grouped_segments 中
  segments = grade_df['Column_8'][row].split('\n')
  #print(len(segments) ," :",segments)

  # 儲存全部課程
  grouped_segments = []
  # 儲存單一課程
  group = []

  for segment in segments:
      # 使用正則表達式匹配數字後緊跟一個大寫字母單詞的位置進行分割
      if re.match(r'\d+\s+[A-Z]', segment):
          grouped_segments.append(group)
          group = []  # 重置group為空列表，準備存儲下一組segments

      group.append(segment)

  # 將剩余的segments加入grouped_segments中
  if group:
      grouped_segments.append(group)
  #print(grouped_segments)

  # 去除逗號
  grouped_grades = ([' '.join(segment)  for segment in grouped_segments])
  # 以空白分隔
  grouped_grades = [segment.split() for segment in grouped_grades]
  grouped_grades = [segment for segment in grouped_grades if len(segment) >= 37]
  #print(grouped_grades)

  #................................................................................................
  # 增加新行: 將每個學期的課程再最後加入行中
  num_rows = len(grouped_grades)
  original_num_rows = grade_df.shape[0]
  for i in range(0, num_rows):
      # 假設要添加新行的索引為 index，其他列的值為 empty_value
      index = grade_df.shape[0]
      empty_value = ''

      # 新行數據
      #new_row = {'Column_8': ','.join(grouped_grades[i]), 'A+': empty_value}

      # 使用 loc 方法添加新行數據，並將其他列的值設為空
      grade_df.loc[index] = empty_value
      grade_df.loc[index, ['Column_8', '學期&學年']] = [','.join(grouped_grades[i]), grade_df.iloc[row,1]]

    #   grade_df = grade_df.append(pd.Series(), ignore_index=True) #.replace(grouped_grades[i+1])
    #   grade_df.iloc[original_num_rows+i-1, 0] =  ','.join(grouped_grades[i])
    #   grade_df.iloc[original_num_rows+i-1, 1] =  grade_df.iloc[row,1] #之後0要改

  # 定義函數，根據條件選擇正確的索引
  def get_index(segment):
      if len(segment[1]) == 1:
          return 2, 14, 3, 16, 32, 17, 33, 18, 34, 19, 35, 20, 36, 21, 37, 22, 23
      # 二次退選
      elif segment[3] == '二次退選':
          return 1, 13, 2, 16, 32, 17, 33, 18, 34, 19, 35, 20, 36, 21, 37, 22, 23
      # 期末評量未填寫
      elif segment[3] == '期末評量未':
          return 1, 17, 2, 19, 35, 20, 36, 21, 37, 22, 38, 23, 39, 24, 40, 25, 26
      else:
          return 1, 13, 2, 15, 31, 16, 32, 17, 33, 18, 34, 19, 35, 20, 36, 21, 22

  # 將數據存入新行
  for i in range(0, len(grouped_grades)):
      #print(grouped_grades)
      i1, i2, i3, i4, i5, i6, i7, i8, i9, i10, i11, i12, i13, i14, i15, i16, i17  = get_index(grouped_grades[i])
      if len(grouped_grades[i]) > i2:
         # print("i2:", i2, "Length of grouped_grades[i]:", len(grouped_grades[i]), " 應該是名稱: ", grouped_grades[i][i2])
          grade_df.iloc[original_num_rows + i, 2] = grouped_grades[i][i1]  # 課程代碼
          grade_df.iloc[original_num_rows + i, 3] = grouped_grades[i][i2]  # 課程名稱
          grade_df.iloc[original_num_rows + i, 4] = grouped_grades[i][i3]  # 學分數
          grade_df.iloc[original_num_rows + i, 6] = grouped_grades[i][i4]  # A+
          grade_df.iloc[original_num_rows + i, 7] = grouped_grades[i][i5]  # A
          grade_df.iloc[original_num_rows + i, 8] = grouped_grades[i][i6]  # A-
          grade_df.iloc[original_num_rows + i, 9] = grouped_grades[i][i7]  # B+
          grade_df.iloc[original_num_rows + i, 10] = grouped_grades[i][i8]  ## B
          grade_df.iloc[original_num_rows + i, 11] = grouped_grades[i][i9]  # B-
          grade_df.iloc[original_num_rows + i, 12] = grouped_grades[i][i10]  # C+
          grade_df.iloc[original_num_rows + i, 13] = grouped_grades[i][i11]  # C
          grade_df.iloc[original_num_rows + i, 14] = grouped_grades[i][i12]  # C-
          grade_df.iloc[original_num_rows + i, 15] = grouped_grades[i][i13]  # D
          grade_df.iloc[original_num_rows + i, 16] = grouped_grades[i][i14]  # E
          grade_df.iloc[original_num_rows + i, 17] = grouped_grades[i][i15]  # X
          grade_df.iloc[original_num_rows + i, 18] = grouped_grades[i][i16]  # 成績未到
          grade_df.iloc[original_num_rows + i, 19] = grouped_grades[i][i17]  # Total

          #print(row, ": ", grouped_grades[i])
      else:
          print("Error: Length of grouped_grades[i] is too short.")


grade_df

,Column_8,學期&學年,課程代碼,課程名稱,學分數,人數/百分比,A+,A,A-,B+,B,B-,C+,C,C-,D,E,X,成績未到,Total
0,1 CC1257303 3 A+ 百分比 20% 36% 1.3% 2.7% 2.7% --...,1081,,,,,,,,,,,,,,,,,,
1,1 CC1255304 3 A+ 百分比 18.1% 29.9% 5.6% .6% 2.3%...,1082,,,,,,,,,,,,,,,,,,
2,1 CC1011301 0 通過 百分比 未通過\n校定英文能力檢測 (人數) 548 54...,1091,,,,,,,,,,,,,,,,,,
3,1 BA305B001 3 A+ 百分比 55.2% 13.4% 4.5% 1.5% -- ...,1092,,,,,,,,,,,,,,,,,,
4,1 B CEG321301 2 A+ 百分比 34.3% 25.9% 5.4% .6% --...,1101,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"7,MI4001301,3,A,百分比,12.3%,27.4%,--,--,--,--,--...",1102,MI4001301,系統分析與設計,3,,9,41,20,2,0,0,0,0,0,1,0,0,0,73
63,"8,PE128B013,0,A+,百分比,20.8%,25%,--,--,--,4.2%,-...",1102,PE128B013,體育(足球)(下),0,,5,11,6,0,0,0,0,0,0,0,1,1,0,24
64,"1,BA2505701,3,A+,百分比,58.3%,33.3%,--,--,--,--,-...",1111,BA2505701,管理數學,3,,7,1,4,0,0,0,0,0,0,0,0,0,0,12
65,"2,MI4113701,3,A+,百分比,33.3%,25%,--,--,--,--,--,...",1111,MI4113701,資料庫交易處理理論與實務,3,,4,4,3,1,0,0,0,0,0,0,0,0,0,12


In [88]:
grade_df.loc[50:]

,Column_8,學期&學年,課程代碼,課程名稱,學分數,人數/百分比,A+,A,A-,B+,B,B-,C+,C,C-,D,E,X,成績未到,Total
50,"3,MB2503301,3,A,百分比,26.5%,35.3%,--,--,--,--,--...",1101,MB2503301,區塊鏈與數據分析,3,,9,9,12,4,0,0,0,0,0,0,0,0,0,34
51,"4,MB499A001,2,A+,百分比,70%,3.3%,--,--,--,--,--,1...",1101,MB499A001,實務專題(上),2,,21,8,1,0,0,0,0,0,0,0,0,0,0,30
52,"5,MI3002301,3,A+,百分比,48.7%,11.5%,7.7%,--,5.1%,...",1101,MI3002301,資料結構,3,,38,12,9,3,6,3,0,2,4,1,0,0,0,78
53,"6,MI4202701,3,A,百分比,19%,14.3%,4.8%,4.8%,4.8%,4...",1101,MI4202701,離散數學,3,,8,9,6,4,2,4,2,2,2,1,2,0,0,42
54,"7,MI4206301,3,A,百分比,13.5%,9.5%,13.5%,6.8%,8.1%...",1101,MI4206301,資訊網路,3,,10,10,7,7,10,9,5,7,6,3,0,0,0,74
55,"8,PE128A012,0,A,百分比,48%,24%,--,--,--,--,--,100...",1101,PE128A012,體育(足球)(上),0,,12,6,6,1,0,0,0,0,0,0,0,0,0,25
56,"1,B,IBG006301,2,A+,百分比,76.5%,5.5%,3.3%,1.1%,.5...",1102,IBG006301,專利101：專利制度入門與基礎,2,,140,11,10,8,6,1,2,2,1,1,1,0,0,183
57,"2,MB499B001,2,A,百分比,58.6%,10.3%,--,--,--,--,--...",1102,MB499B001,實務專題(下),2,,17,9,3,0,0,0,0,0,0,0,0,0,0,29
58,"3,MI3003301,3,A+,百分比,45.7%,11.4%,5.7%,4.3%,--,...",1102,MI3003301,管理資訊系統,3,,32,20,8,3,4,0,3,0,0,0,0,0,0,70
59,"4,MI3103701,3,A+,百分比,10.7%,21.3%,12%,2.7%,8%,8...",1102,MI3103701,演算法,3,,8,9,16,8,9,5,2,5,6,1,6,0,0,75


In [89]:
print("i2:", i2, "Length of grouped_grades[i]:", len(grouped_grades[i]))

i2: 14 Length of grouped_grades[i]: 39


## dataframe整理

In [90]:
grade_df.insert(loc=4, column='教師名稱', value="")

In [91]:
grade_df.drop([0, 1, 2, 3, 4, 5, 6], inplace=True, axis = 0)
grade_df.reset_index(drop=True, inplace=True)
grade_df.drop('Column_8', inplace=True, axis = 1)
grade_df['人數/百分比'] = ''

In [92]:
# 按照學期&學年和課程代碼排序
grade_df = grade_df.sort_values(by=['學期&學年', '課程代碼'], ascending=[False, True])
# 重置索引，生成新的序号
grade_df.reset_index(drop=True, inplace=True)

# 顯示排序後的 DataFrame
grade_df

,學期&學年,課程代碼,課程名稱,教師名稱,學分數,人數/百分比,A+,A,A-,B+,B,B-,C+,C,C-,D,E,X,成績未到,Total
0,1111,BA2505701,管理數學,,3,,7,1,4,0,0,0,0,0,0,0,0,0,0,12
1,1111,MI4113701,資料庫交易處理理論與實務,,3,,4,4,3,1,0,0,0,0,0,0,0,0,0,12
2,1111,MIG005301,大數據分析導論,,3,,8,20,22,1,1,0,0,0,0,0,0,0,0,52
3,1102,IBG006301,專利101：專利制度入門與基礎,,2,,140,11,10,8,6,1,2,2,1,1,1,0,0,183
4,1102,MB499B001,實務專題(下),,2,,17,9,3,0,0,0,0,0,0,0,0,0,0,29
5,1102,MI3003301,管理資訊系統,,3,,32,20,8,3,4,0,3,0,0,0,0,0,0,70
6,1102,MI3103701,演算法,,3,,8,9,16,8,9,5,2,5,6,1,6,0,0,75
7,1102,MI3114301,行動裝置程式設計,,3,,30,13,6,4,4,3,3,4,4,1,5,2,0,79
8,1102,MI3201301,作業研究(一),,3,,23,9,8,0,3,3,5,5,3,2,7,0,0,68
9,1102,MI4001301,系統分析與設計,,3,,9,41,20,2,0,0,0,0,0,1,0,0,0,73


# 3.查詢授課教師(爬蟲台科課程查詢網站: https://querycourse.ntust.edu.tw/querycourse/#/ )

In [93]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [94]:
pip install selenium

In [95]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

def WebDriver(semester, course_code):
        # 初始化 Chrome WebDriver
        driver = webdriver.Chrome()

        # 打開網頁
        driver.get('https://querycourse.ntust.edu.tw/querycourse/#/')

        # 點擊學年期按鈕 <div class="v-menu__activator"><button type="button" class="subheading v-btn v-btn--flat theme--dark"><div class="v-btn__content"> 學年期 ： 1122
        # 使用顯式等待來等待學年期按鈕出現
        semester_button = driver.find_element(By.XPATH, '//div[contains(@class, "v-menu__activator")]//button[contains(@class, "subheading v-btn v-btn--flat theme--dark")]' )#and contains(@class, "v-btn") and contains(@class, "v-btn--flat") and contains(@class, "theme--dark")]')
        semester_button.send_keys(Keys.ENTER)


        # 在彈出的學年選擇框中選擇特定的學年（假設是1101） <div class="v-menu__content theme--light " style="min-width: 0px; top: 0px; left: 0px; transform-origin: left top; z-index: 0; display: none;"><div role="list" class="v-list theme--light"><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1122</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1121</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">111H</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1112</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1111</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">110H</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1102</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1101</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">109H</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1092</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1091</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">108H</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1082</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1081</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">107H</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1072</div></a></div><div role="listitem"><a class="v-list__tile v-list__tile--link theme--light"><div class="v-list__tile__title">1071</div></a></div></div></div>
        # 等待元素可點擊後再進行點擊
        # semester = "1101"
        # wait = WebDriverWait(driver, 10)
        # element = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='v-list__tile__title' and text()='{semester}']")))#1101
        # # 執行 JavaScript 來點擊元素
        # driver.execute_script("arguments[0].click();", element)
        #semester = str(1101)  # 请注意，这里的 semester 是字符串类型
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[@class='v-list__tile__title' and text()='{semester}']")))
        driver.execute_script("arguments[0].click();", element)


        # 輸入查詢的課程代號
        #course_code = 'MI3002301'
        course_code_input = driver.find_element(By.CSS_SELECTOR, 'input[aria-label="課程代碼"]')
        course_code_input.send_keys(course_code)  # 替換成您要查詢的課程代號 #MI3002301

        # 點擊查詢按鈕
        # 查詢的button: <button type="button" class="v-btn v-btn--block v-btn--round theme--light"><div class="v-btn__content"><i aria-hidden="true" class="v-icon material-icons theme--light">search</i>
        search_button = driver.find_element(By.CSS_SELECTOR, 'button.v-btn.v-btn--block.v-btn--round.theme--light')
        search_button.send_keys(Keys.ENTER)

        # # 等待查詢結果加載完成
        wait = WebDriverWait(driver, 30)

        #  提取授課教師的值 <table class="v-datatable v-table theme--light"><thead><tr><th role="columnheader" scope="col" width="7%" aria-label="課程代碼: Not sorted." aria-sort="none" class="column text-xs-center">課程代碼</th><th role="columnheader" scope="col" width="10%" aria-label="課程名稱: Not sorted." aria-sort="none" class="column text-xs-center">課程名稱</th><th role="columnheader" scope="col" width="7%" aria-label="授課教師: Not sorted." aria-sort="none" class="column text-xs-center">授課教師</th><th role="columnheader" scope="col" width="7%" aria-label="上課星期節次: Not sorted." aria-sort="none" class="column text-xs-center">上課星期節次</th><th role="columnheader" scope="col" width="7%" aria-label="教室: Not sorted." aria-sort="none" class="column text-xs-center">教室</th><th role="columnheader" scope="col" width="3%" aria-label="大綱: Not sorted." aria-sort="none" class="column text-xs-center">大綱</th></tr><tr class="v-datatable__progress"><th colspan="6" class="column"></th></tr></thead><tbody><tr><td class="" style="padding: 0px 5px;">GE3604301</td><td style="padding: 0px 5px;">性別與法律</td><td style="padding: 0px 5px;">張宏誠</td><td style="padding: 0px 5px; word-break: break-all;">F6,F7
            #</td><td style="padding: 0px 5px;">TR-413-2</td><td style="padding: 0px 5px;"><button type="button" class="v-btn v-btn--flat v-btn--icon theme--light"><div class="v-btn__content"><i aria-hidden="true" class="v-icon material-icons theme--light">more_horiz</i></div></button></td><!----></tr></tbody></table>
        # 使用WebDriverWait來等待授課教師的value
        teacher_element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//table[@class="v-datatable v-table theme--light"]/tbody/tr/td[7]'))
        )
        teacher_name = teacher_element.text
        #print("授課教師:", teacher_name)

    
        # 關閉瀏覽器
        driver.quit()
        
        return teacher_name

In [96]:
for row in range (0, 5): # 由於部分學期太久遠，跑的時間較久，先跑前面即可 #0,grade_df.shape[0]
   grade_df.iloc[row, 3] = WebDriver(grade_df.iloc[row, 0], grade_df.iloc[row, 1])

In [97]:
print(grade_df.iloc[:20, :])

   學期&學年       課程代碼             課程名稱 教師名稱 學分數 人數/百分比   A+   A  A-  B+   B B-  \
0   1111  BA2505701             管理數學  王聖淵   3           7   1   4   0   0  0   
1   1111  MI4113701     資料庫交易處理理論與實務  呂永和   3           4   4   3   1   0  0   
2   1111  MIG005301          大數據分析導論  張智傑   3           8  20  22   1   1  0   
3   1102  IBG006301  專利101：專利制度入門與基礎  管中徽   2         140  11  10   8   6  1   
4   1102  MB499B001          實務專題(下)        2          17   9   3   0   0  0   
5   1102  MI3003301           管理資訊系統        3          32  20   8   3   4  0   
6   1102  MI3103701              演算法        3           8   9  16   8   9  5   
7   1102  MI3114301         行動裝置程式設計        3          30  13   6   4   4  3   
8   1102  MI3201301          作業研究(一)        3          23   9   8   0   3  3   
9   1102  MI4001301          系統分析與設計        3           9  41  20   2   0  0   
10  1102  PE128B013        體育(足球)(下)        0           5  11   6   0   0  0   
11  1101  CEG321301        人文與科技實務講座    

資料庫 Database

In [98]:
pip install mysql

Note: you may need to restart the kernel to use updated packages.


In [99]:
pip install mysql-connector-python

建立資料庫

In [100]:
import sqlite3
import pandas as pd


# 連接到SQLite數據庫（如果不存在則會創建）
conn = sqlite3.connect('ntust_grades.db')

# 將DataFrame存儲到數據庫中，表名為grades
grade_df.to_sql('grades', conn, index=False, if_exists='replace')

# 關閉數據庫連接
conn.close()

DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': database is locked

更新資料庫

In [ ]:
import sqlite3
import pandas as pd


# 連接到SQLite數據庫（如果不存在將會創建）
conn = sqlite3.connect('ntust_grades.db')

# 將DataFrame存儲到數據庫中，表名為grades
grade_df.to_sql('grades', conn, index=False, if_exists='replace')
import sqlite3
import time

# 創建數據庫連接的上下文管理器
class DBConnection:
    def __enter__(self):
        self.conn = sqlite3.connect('ntust_grades.db')
        return self.conn
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.close()

# 將 DataFrame 中的數據更新到數據庫中
def update_database(grade_df):
    retry_limit = 3  # 設置重試次數上限
    retry_count = 0
    while retry_count < retry_limit:
        try:
            with DBConnection() as conn:
                cursor = conn.cursor()
                for index, row in grade_df.iterrows():
                    teacher_name = row['教師名稱']
                    sql = f"INSERT INTO grades (教師名稱) VALUES ('{teacher_name}')"
                    cursor.execute(sql)
                conn.commit()
            print("數據更新成功！")
            break
        except sqlite3.OperationalError as e:
            print(f"發生數據庫鎖定錯誤：{e}")
            retry_count += 1
            print(f"第 {retry_count} 次重試...")
            time.sleep(1)  # 暫停一秒後重試
    else:
        print("達到重試次數上限，數據更新失敗！")

# 調用函數更新數據庫
update_database(grade_df)
# 關閉數據庫連接
conn.close()

发生数据库锁定错误：database is locked
第 1 次重试...
发生数据库锁定错误：database is locked
第 2 次重试...
发生数据库锁定错误：database is locked
第 3 次重试...
达到重试次数上限，数据更新失败！


In [ ]:
import sqlite3

# 創建數據庫連接
conn = sqlite3.connect('ntust_grades.db')

# 將 DataFrame 中的數據更新到數據庫中
for index, row in grade_df.iterrows():
    # 獲取教師名稱的值
    teacher_name = row['教師名稱']
    
    # 構造 SQL 插入語句
    sql = f"INSERT INTO grades (教師名稱) VALUES ('{teacher_name}')"
    
    # 執行 SQL 插入操作
    conn.execute(sql)

# 提交並關閉連接
conn.commit()
conn.close()

OperationalError: database is locked

sql 語法

In [ ]:
# 用來連接資料庫，也可以寫sql查找
conn = sqlite3.connect('ntust_grades.db')
cursor = conn.execute('SELECT * from grades') # where "學期&學年" == "1111"
rows = cursor.fetchmany(10) #fetchall()
for row in rows:
    print(row)

('1111', 'BA2505701', '管理數學', '', '3', '', '7', '1', '4', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '12')
('1111', 'MI4113701', '資料庫交易處理理論與實務', '', '3', '', '4', '4', '3', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '12')
('1111', 'MIG005301', '大數據分析導論', '', '3', '', '8', '20', '22', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '52')
('1102', 'IBG006301', '專利101：專利制度入門與基礎', '', '2', '', '140', '11', '10', '8', '6', '1', '2', '2', '1', '1', '1', '0', '0', '183')
('1102', 'MB499B001', '實務專題(下)', '', '2', '', '17', '9', '3', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '29')
('1102', 'MI3003301', '管理資訊系統', '', '3', '', '32', '20', '8', '3', '4', '0', '3', '0', '0', '0', '0', '0', '0', '70')
('1102', 'MI3103701', '演算法', '', '3', '', '8', '9', '16', '8', '9', '5', '2', '5', '6', '1', '6', '0', '0', '75')
('1102', 'MI3114301', '行動裝置程式設計', '', '3', '', '30', '13', '6', '4', '4', '3', '3', '4', '4', '1', '5', '2', '0', '79')
('1102', 'MI3201301', '作業研究(一)', '', '3', '',